In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!cp /content/gdrive/My\ Drive/Task1/input.csv input.csv

In [4]:
#read in the sample file
input_csv = pd.read_csv("input.csv")

input_csv.head()

,bins_lat,"(6.061, 6.219]","(6.219, 6.367]","(6.367, 6.516]","(6.516, 6.665]","(6.665, 6.813]","(6.813, 6.962]","(6.962, 7.11]","(7.11, 7.259]","(7.259, 7.408]","(7.408, 7.556]","(7.556, 7.705]","(7.705, 7.853]","(7.853, 8.002]","(8.002, 8.151]","(8.151, 8.299]","(8.299, 8.448]","(8.448, 8.596]","(8.596, 8.745]","(8.745, 8.894]","(8.894, 9.042]","(9.042, 9.191]","(9.191, 9.339]","(9.339, 9.488]","(9.488, 9.637]","(9.637, 9.785]","(9.785, 9.934]","(9.934, 10.082]","(10.082, 10.231]","(10.231, 10.38]","(10.38, 10.528]","(10.528, 10.677]","(10.677, 10.825]","(10.825, 10.974]","(10.974, 11.123]","(11.123, 11.271]","(11.271, 11.42]","(11.42, 11.568]","(11.568, 11.717]","(11.717, 11.866]","(11.866, 12.014]","(12.014, 12.163]","(12.163, 12.311]","(12.311, 12.46]","(12.46, 12.609]","(12.609, 12.757]","(12.757, 12.906]","(12.906, 13.054]","(13.054, 13.203]","(13.203, 13.352]","(13.352, 13.5]","(13.5, 13.649]","(13.649, 13.797]","(13.797, 13.946]","(13.946, 14.095]","(14.095, 14.243]","(14.243, 14.392]","(14.392, 14.54]","(14.54, 14.689]","(14.689, 14.838]","(14.838, 14.986]"
0,"(47.511, 47.601]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1104.8,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(47.601, 47.684]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1403.78,0.00,1900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1183.2,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1600.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(47.684, 47.766]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5300.0,0.0,0.0,0.00,2134.44,0.0,0.0,0.0,1863.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8730.0,1200.0,0.0,8800.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(47.766, 47.849]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,12486.0,4300.0,0.0,0.0,0.0,0.0,0.0,5000.0,0.0,5220.0,0.0,0.0,8624.0,2014.80,1601.36,0.0,3150.0,0.0,1412.84,0.0,0.0,0.0,0.0,0.0,1206.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"(47.849, 47.932]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1782.00,0.00,0.0,0.0,0.0,17000.0,0.0,8400.0,0.0,0.0,4675.0,0.0,4800.0,8500.0,0.0,0.0,0.0,6743.31,2790.00,3194.0,0.0,0.0,0.00,0.0,0.0,0.0,1725.0,0.0,1375.0,0.0,5929.4,1300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#Transpose columns to lines
df = input_csv.melt(id_vars="bins_lat")
df.head()

,bins_lat,variable,value
0,"(47.511, 47.601]","(6.061, 6.219]",0.0
1,"(47.601, 47.684]","(6.061, 6.219]",0.0
2,"(47.684, 47.766]","(6.061, 6.219]",0.0
3,"(47.766, 47.849]","(6.061, 6.219]",0.0
4,"(47.849, 47.932]","(6.061, 6.219]",0.0


In [6]:
#select none 0 lines only
df = df.loc[df['value']!=0]
#rename variables
df = df.rename(columns = {'bins_lat':'lat','variable':'long','value':'factor'})
df.head()

,lat,long,factor
39,"(50.743, 50.825]","(6.061, 6.219]",7720.00
40,"(50.825, 50.908]","(6.061, 6.219]",8294.00
45,"(51.239, 51.321]","(6.061, 6.219]",1500.00
119,"(49.916, 49.998]","(6.219, 6.367]",2247.30
121,"(50.081, 50.164]","(6.219, 6.367]",3705.75


In [7]:
# Split original geocodes
newLat  = df["lat" ].str.split(", ", n = 1, expand = True)
newLong = df["long"].str.split(", ", n = 1, expand = True)
df['lat1'] = newLat[0].str.replace("(","").astype(float)
df['lat2'] = newLat[1].str.replace("]","").astype(float)
df['long1'] = newLong[0].str.replace("(","").astype(float)
df['long2'] = newLong[1].str.replace("]","").astype(float)
df['lat_avg'] = (df['lat1'] + df['lat2'])/2
df['long_avg'] = (df['long1'] + df['long2'])/2
df['lat_dif'] = df['lat2'] - df['lat1']
df['long_dif'] = df['long2'] - df['long1']

#normalize factor
mean_df = df.mean()
std_df = df.std()
df['factor_norm'] = (df['factor'] - mean_df['factor']) / std_df['factor']
df.head()

,lat,long,factor,lat1,lat2,long1,long2,lat_avg,long_avg,lat_dif,long_dif,factor_norm
39,"(50.743, 50.825]","(6.061, 6.219]",7720.00,50.743,50.825,6.061,6.219,50.7840,6.140,0.082,0.158,0.267397
40,"(50.825, 50.908]","(6.061, 6.219]",8294.00,50.825,50.908,6.061,6.219,50.8665,6.140,0.083,0.158,0.343802
45,"(51.239, 51.321]","(6.061, 6.219]",1500.00,51.239,51.321,6.061,6.219,51.2800,6.140,0.082,0.158,-0.560541
119,"(49.916, 49.998]","(6.219, 6.367]",2247.30,49.916,49.998,6.219,6.367,49.9570,6.293,0.082,0.148,-0.461068
121,"(50.081, 50.164]","(6.219, 6.367]",3705.75,50.081,50.164,6.219,6.367,50.1225,6.293,0.083,0.148,-0.266935


In [8]:
# import packages to present the data
!pip install plotly_express
import plotly_express as px

In [10]:
px.scatter_geo(df, lat='lat1', lon='long1',size='factor',scope='europe',color='factor_norm',title="Germany", opacity=0.6,width= 600 , height=450)